In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
from tensorflow.contrib.eager.python import tfe
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

In [2]:
problems.available()

['algorithmic_addition_binary40',
 'algorithmic_addition_decimal40',
 'algorithmic_cipher_shift200',
 'algorithmic_cipher_shift5',
 'algorithmic_cipher_vigenere200',
 'algorithmic_cipher_vigenere5',
 'algorithmic_identity_binary40',
 'algorithmic_identity_decimal40',
 'algorithmic_multiplication_binary40',
 'algorithmic_multiplication_decimal40',
 'algorithmic_reverse_binary40',
 'algorithmic_reverse_binary40_test',
 'algorithmic_reverse_decimal40',
 'algorithmic_reverse_nlplike32k',
 'algorithmic_reverse_nlplike8k',
 'algorithmic_shift_decimal40',
 'audio_timit_characters_tune',
 'audio_timit_tokens8k_test',
 'audio_timit_tokens8k_tune',
 'gym_pong_random5k',
 'gym_pong_trajectories_from_policy',
 'image_celeba',
 'image_cifar10',
 'image_cifar100',
 'image_cifar100_plain',
 'image_cifar100_plain8',
 'image_cifar100_plain_gen',
 'image_cifar100_tune',
 'image_cifar10_plain',
 'image_cifar10_plain8',
 'image_cifar10_plain_gen',
 'image_cifar10_tune',
 'image_fashion_mnist',
 'image_fsn

In [3]:
# Fetch the MNIST problem
ptb_problem = problems.problem("languagemodel_ptb10k")
# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
ptb_problem.generate_data(data_dir, tmp_dir)

INFO:tensorflow:Not downloading, file already found: /home/mtdesta/t2t/tmp/simple-examples.tgz


[2018-04-03 15:25:10,913] Not downloading, file already found: /home/mtdesta/t2t/tmp/simple-examples.tgz


INFO:tensorflow:Skipping generator because outputs files exist


[2018-04-03 15:25:11,724] Skipping generator because outputs files exist


INFO:tensorflow:Not downloading, file already found: /home/mtdesta/t2t/tmp/simple-examples.tgz


[2018-04-03 15:25:11,725] Not downloading, file already found: /home/mtdesta/t2t/tmp/simple-examples.tgz


INFO:tensorflow:Skipping generator because outputs files exist


[2018-04-03 15:25:12,473] Skipping generator because outputs files exist


INFO:tensorflow:Skipping shuffle because output files exist


[2018-04-03 15:25:12,475] Skipping shuffle because output files exist


In [4]:
registry.list_models()

['aligned',
 'attention_lm',
 'attention_lm_moe',
 'basic_autoencoder',
 'basic_conv_gen',
 'basic_discrete_autoencoder',
 'basic_fc_relu',
 'byte_net',
 'cycle_gan',
 'diagonal_neural_gpu',
 'gene_expression_conv',
 'imagetransformer',
 'imagetransformer2d',
 'imagetransformer_moe',
 'img2img_transformer',
 'lstm_encoder',
 'lstm_seq2seq',
 'lstm_seq2seq_attention',
 'lstm_seq2seq_attention_bidirectional_encoder',
 'lstm_seq2seq_bidirectional_encoder',
 'multi_model',
 'neural_gpu',
 'resnet',
 'revnet',
 'shake_shake',
 'slice_net',
 'super_lm',
 'transformer',
 'transformer_ae',
 'transformer_encoder',
 'transformer_moe',
 'transformer_revnet',
 'transformer_sketch',
 'transformer_symshard',
 'vanilla_gan',
 'xception']

In [5]:
#hparams.hidden_size = 64
model_name = "transformer"
hparams_set = "transformer_small"


hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="languagemodel_ptb10k")
model = registry.model(model_name)(hparams, Modes.TRAIN)

[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', ('symbol', 10000)), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Setting T2TModel mode to 'train'


[2018-04-03 15:25:12,539] Setting T2TModel mode to 'train'


In [6]:
@tfe.implicit_value_and_gradients
def loss_fn(features):
  _, losses = model(features)
  return losses["training"]

BATCH_SIZE = 32
max_num_of_words = 150
ptb_train_dataset = ptb_problem.dataset(Modes.TRAIN, data_dir)
ptb_train_dataset.output_shapes["targets"] = [max_num_of_words]
ptb_train_dataset = ptb_problem.dataset(Modes.TRAIN, data_dir)
ptb_train_dataset = ptb_train_dataset.repeat(None).padded_batch(BATCH_SIZE, ptb_train_dataset.output_shapes)

INFO:tensorflow:Reading data files from /home/mtdesta/t2t/data/languagemodel_ptb10k-train*


[2018-04-03 15:25:12,570] Reading data files from /home/mtdesta/t2t/data/languagemodel_ptb10k-train*


INFO:tensorflow:partition: 0 num_data_files: 10


[2018-04-03 15:25:12,578] partition: 0 num_data_files: 10


INFO:tensorflow:Reading data files from /home/mtdesta/t2t/data/languagemodel_ptb10k-train*


[2018-04-03 15:25:13,175] Reading data files from /home/mtdesta/t2t/data/languagemodel_ptb10k-train*


INFO:tensorflow:partition: 0 num_data_files: 10


[2018-04-03 15:25:13,180] partition: 0 num_data_files: 10


In [7]:
optimizer = tf.train.AdamOptimizer()

In [8]:
print(ptb_train_dataset)

<PaddedBatchDataset shapes: {targets: (?, ?)}, types: {targets: tf.int64}>


In [9]:
NUM_STEPS =10

for count, example in enumerate(tfe.Iterator(ptb_train_dataset)):
  example["targets"] = tf.reshape(example["targets"], [BATCH_SIZE, -1, 1, 1])  # Make it 4D.
  loss, gv = loss_fn(example)
  optimizer.apply_gradients(gv)

  if count % 50 == 0:
    print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  if count >= NUM_STEPS:
    break

INFO:tensorflow:Using variable initializer: uniform_unit_scaling


[2018-04-03 15:25:13,288] Using variable initializer: uniform_unit_scaling


[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Transforming 'targets' with symbol_modality_10000_256.targets_bottom


[2018-04-03 15:25:13,291] Transforming 'targets' with symbol_modality_10000_256.targets_bottom


INFO:tensorflow:Building model body


...

[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Using variable initializer: uniform_unit_scaling


[2018-04-03 15:25:20,066] Using variable initializer: uniform_unit_scaling


[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Using variable initializer: uniform_unit_scaling


[2018-04-03 15:25:20,836] Using variable initializer: uniform_unit_scaling


[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]


In [10]:
model.set_mode(Modes.EVAL)
ptb_eval_dataset = ptb_problem.dataset(Modes.EVAL, data_dir)

# Create eval metric accumulators for accuracy (ACC) and accuracy in
# top 5 (ACC_TOP5)
metrics_accum, metrics_result = metrics.create_eager_metrics(
    [metrics.Metrics.NEG_LOG_PERPLEXITY, metrics.Metrics.APPROX_BLEU])

for count, example in enumerate(tfe.Iterator(ptb_eval_dataset)):
  if count >= 200:
    break

  # Make the inputs and targets 4D
  #example["inputs"] = tf.reshape(example["inputs"], [1, -1, 32, 1])
  example["targets"] = tf.reshape(example["targets"], [1, -1, 1, 1])

  # Call the model
  predictions, _ = model(example)

  # Compute and accumulate metrics
  metrics_accum(predictions, example["targets"])

# Print out the averaged metric values on the eval data
for name, val in metrics_result().items():
  print("%s: %.2f" % (name, val))

INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-04-03 15:25:21,604] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.symbol_dropout to 0.0


[2018-04-03 15:25:21,606] Setting hparams.symbol_dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Using variable initializer: uniform_unit_scaling


[2018-04-03 15:25:30,396] Using variable initializer: uniform_unit_scaling


[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Using variable initializer: uniform_unit_scaling


[2018-04-03 15:25:30,434] Using variable initializer: uniform_unit_scaling


[('batch_size_multiplier', 1), ('input_modality', {}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', <tensor2tensor.layers.modalities.SymbolModality object at 0x7ff12dce75c0>), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.TokenTextEncoder object at 0x7ff12dce7320>}), ('was_copy', False), ('was_reversed', False)]
approx_bleu_score: 0.04
neg_log_perplexity: -7.08
